# Data

In [2]:
df = pd.read_csv('chembl.csv')

/data/home/viktor_petukhov/local/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### GANs

In [3]:
num_elems = len(elems)

In [4]:
rnn_cell_size = 128

#### Descriminator

In [5]:
X = tf.placeholder(tf.float32, shape=[None, None, num_elems], name='Discr_Input')

true_lengths = tf.placeholder(tf.int32, shape=[None], name='Lengths')

In [9]:
forward_cell_descr.variables

[]

In [6]:
forward_cell_descr = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)
backwards_cell_descr = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)

D_W_1 = tf.get_variable('D_W_1', shape=[rnn_cell_size*2, rnn_cell_size], initializer=xavier_initializer(), dtype=tf.float64)
D_B_1 = tf.get_variable('D_B_1', shape=[rnn_cell_size], initializer=tf.zeros_initializer(), dtype=tf.float64)

D_W_2 = tf.get_variable('D_W_2', shape=[rnn_cell_size, 1], initializer=xavier_initializer(), dtype=tf.float64)
D_B_2 = tf.get_variable('D_B_2', shape=[1], initializer=tf.zeros_initializer(), dtype=tf.float64)

descriminator_weights = [D_W_1, D_W_2, D_B_1, D_B_2, ]

In [11]:
forward_cell_descr.zero_state()

TypeError: zero_state() missing 2 required positional arguments: 'batch_size' and 'dtype'

In [7]:
def descriminator(X, lengths):
    
    # Bi-RNN
    
    _, (fw_state, bw_state) = tf.nn.bidirectional_dynamic_rnn(forward_cell_descr, 
                                                             backwards_cell_descr, X, 
                                                             sequence_length=lengths, dtype=tf.float32)

    sequence_description = tf.concat([fw_state, bw_state], axis=-1)
    
    # First dense
    
    f_d = tf.nn.relu(tf.matmul(sequence_description, D_W_1) + D_B_1)
    
    # Output
    
    return tf.sigmoid(tf.matmul(f_d, D_W_2) + D_B_2)

In [8]:
true_prob = descriminator(X, true_lengths)

#### Generator

In [9]:
def sample_Z(m, n):
    return np.random.uniform(-1, 1, [m, n])

In [10]:
max_length = 500

In [15]:
Z = tf.placeholder(tf.float32, shape=[None, max_length, 1], name='Z_Input')
T = tf.Variable(100, trainable=False)

In [ ]:
forward_cell_gen = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)
backwards_cell_gen = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)

G_W_1 = tf.get_variable('G_W_1', shape=[rnn_cell_size*2, rnn_cell_size], initializer=xavier_initializer(), dtype=tf.float64)
G_B_1 = tf.get_variable('G_B_1', shape=[rnn_cell_size], initializer=tf.zeros_initializer(), dtype=tf.float64)

G_W_2 = tf.get_variable('G_W_2', shape=[rnn_cell_size, len(elems)], initializer=xavier_initializer(), dtype=tf.float64)
G_B_2 = tf.get_variable('G_B_2', shape=[len(elems)], initializer=tf.zeros_initializer(), dtype=tf.float64)

In [ ]:
def generator(Z, T, gumbel=False):
    
    
    (fw_outputs_gen, bw_outputs_gen), _ = tf.nn.bidirectional_dynamic_rnn(forward_cell_gen, backwards_cell_gen, 
                                                                          Z, dtype=tf.float32)
    
    rnn_outputs = tf.concat([fw_outputs, bw_outputs], axis=-1)
    
    flat_repr = tf.reshape(rnn_outputs, [-1, 2*rnn_cell_size])
    
    f_d = tf.nn.relu(tf.matmul(flat_repr, G_W_1) + G_B_1)
    
    flat_output = tf.matmul(f_d, G_W_2) + G_B_2    
    
    output_logits = tf.reshape(flat_output, [-1, max_length, num_elem])
    
    # Discrete outputs

    prob_outputs = tf.nn.softmax(output_logits)

    discrete_outputs = tf.cast(tf.arg_max(prob_outputs, -1), tf.int32)

    one_hots = tf.one_hot(discrete_outputs, len(elems))
    
    # Length determination
    
    endings = tf.cast(tf.argmax(tf.cast(tf.equal(elems.index(end_elem), discrete_outputs), tf.int32), axis=-1), tf.int32)

    def true_length(ind):
        return tf.cond(tf.equal(0, ind), lambda: tf.cast(max_length, tf.int32), lambda: ind)

    generated_lengths = tf.map_fn(true_length, endings)
    
    if gumbel:
        return gumbel_softmax(output_logits, T), generated_lengths
    
    return prob_outputs, generated_lengths

In [ ]:
for_gen_X, for_gen_Len = generator(Z, T, gumbel=True)
for_descr_X, for_descr_Len = generator(Z, T, gumbel=False)

false_prob_descriminator = descriminator(for_descr_X, for_descr_Len)
false_prob_gen = descriminator(for_gen_X, for_gen_Len)

In [16]:
forward_cell_gen = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)
backwards_cell_gen = tf.contrib.rnn.BasicLSTMCell(rnn_cell_size)

(fw_outputs_gen, bw_outputs_gen), _ = tf.nn.bidirectional_dynamic_rnn(forward_cell_gen, backwards_cell_gen, Z, dtype=tf.float32)

ValueError: Variable bidirectional_rnn/fw/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-7-ae181086d401>", line 9, in descriminator
    X, sequence_length=lengths, dtype=tf.float32)
  File "<ipython-input-8-ec9a9d819231>", line 1, in <module>
    true_prob = descriminator(X, true_lengths)
  File "/data/home/viktor_petukhov/local/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
rnn_outputs = tf.concat([fw_outputs, bw_outputs], axis=-1)

NameError: name 'fw_outputs' is not defined

In [150]:
flat_repr = tf.reshape(rnn_outputs, [-1, 2*rnn_cell_size])

In [152]:
flat_output = tf.layers.dense(flat_repr, num_elem, activation=tf.nn.relu, kernel_initializer=xavier_initializer())

In [153]:
output_logits = tf.reshape(flat_output, [-1, max_length, num_elem])

In [157]:
# Discrete ouputs

prob_outputs = tf.nn.softmax(output_logits)

discrete_outputs = tf.cast(tf.arg_max(prob_outputs, -1), tf.int32)

descr_input = tf.one_hot(discrete_outputs, len(elems))

In [390]:
# Outputs for training

endings = tf.cast(tf.argmax(tf.cast(tf.equal(elems.index(end_elem), discrete_outputs), tf.int32), axis=-1), tf.int32)

def true_length(ind):
    return tf.cond(tf.equal(0, ind), lambda: tf.cast(max_length, tf.int32), lambda: ind)

generated_lengths = tf.map_fn(true_length, endings)

In [ ]:
descr_loss = -tf.reduce_mean(tf.log(true_prob) + tf.log(1. - false_prob_descriminator))
gen_loss = -tf.reduce_mean(tf.log(fake_prob_gen))

# Descriminator training

In [ ]:
false_probs = descriminator(descr_input, generated_lengths)

In [ ]:
gumbel_output = gumbel_softmax(output_logits, T)

In [ ]:
gen_prob = descriminator(gumbel_output, generated_lengths)

#### Training

In [411]:
sess = tf.Session()

In [412]:
sess.run(tf.global_variables_initializer())

In [413]:
data_raw = df.canonical_smiles.iloc[:50].apply(to_one_hot).values

lengths = list(map(lambda x: x.shape[0], data_raw))
data = pad_sequences(data_raw, truncating='post')

In [414]:
a, b = sess.run([discrete_outputs, generated_lengths], feed_dict={Z: sample_Z(1000, max_length).reshape([1000, max_length, 1])})

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Z_Input:0", shape=(?, 500, 1), dtype=float32) is not an element of this graph.

In [405]:
for i, x in enumerate(a):
    if len(to_smile(x)) == 500:
        print(len(to_smile(x)))
        print(i)